In [1]:
#export
"""
This is for functions that sort of changes the table
structure in a dramatic way. They're the core transformations
"""
from typing import List, Union, Iterator, Callable, Any, Tuple, Dict
from collections import defaultdict, Counter
from k1lib.bioinfo.cli.init import patchDefaultDelim, BaseCli, oneToMany, T, Table
import k1lib.bioinfo.cli as cli
import itertools
__all__ = ["joinColumns", "transpose", "splitColumns", "joinList", "joinStreams",
           "insertRow", "insertColumn", "insertIdColumn",
           "toDict", "split", "table", "stitch", "listToTable", "tableFromList",
           "count", "permute", "accumulate", "AA_", "sample", "infinite"]

In [2]:
#export
class joinColumns(BaseCli):
    def __init__(self, fillValue=None):
        """Join multiple columns and loop through all rows. Aka transpose.

:param fillValue: if not None, then will try to zip longest with this fill value

Example::

    # returns [[1, 4], [2, 5], [3, 6]]
    [[1, 2, 3], [4, 5, 6]] | joinColumns() | dereference()
    # returns [[1, 4], [2, 5], [3, 6], [0, 7]]
    [[1, 2, 3], [4, 5, 6, 7]] | joinColumns(0) | dereference()"""
        self.fillValue = fillValue
    def __ror__(self, it:Iterator[Iterator[T]]) -> Table[T]:
        if self.fillValue is None: yield from zip(*it)
        else: yield from itertools.zip_longest(*it, fillvalue=self.fillValue)
splitColumns = transpose = joinColumns

In [3]:
assert [[1, 2, 3], [4, 5, 6]] | joinColumns() | cli.dereference() == [[1, 4], [2, 5], [3, 6]]
assert [[1, 2, 3], [4, 5, 6, 7]] | joinColumns(0) | cli.dereference() == [[1, 4], [2, 5], [3, 6], [0, 7]]

In [4]:
#export
class joinList(BaseCli):
    def __init__(self, element=None, begin=True):
        """Join element into list.

:param element: the element to insert. If None, then takes the input [e, [...]],
    else takes the input [...] as usual

Example::

    # returns [5, 2, 6, 8]
    [5, [2, 6, 8]] | joinList()
    # also returns [5, 2, 6, 8]
    [2, 6, 8] | joinList(5)
"""
        self.element = element; self.begin = begin
    def __ror__(self, it:Tuple[T, Iterator[T]]) -> Iterator[T]:
        it = iter(it)
        if self.element is None:
            if self.begin: yield next(it); yield from next(it)
            else: e = next(it); yield from next(it); yield e
        else:
            if self.begin: yield self.element; yield from it
            else: yield from it; yield self.element

In [5]:
assert [5, [2, 6, 8]] | joinList() | cli.dereference() == [5, 2, 6, 8]
assert [2, 6, 8] | joinList(5) | cli.dereference() == [5, 2, 6, 8]

In [6]:
#export
class joinStreams(BaseCli):
    """Join multiple streams.
Example::

    # returns [1, 2, 3, 4, 5]
    [[1, 2, 3], [4, 5]] | joinStreams() | dereference()
"""
    def __ror__(self, streams:Iterator[Iterator[T]]) -> Iterator[T]:
        for stream in streams: yield from stream

In [7]:
assert [[1, 2, 3], [4, 5]] | joinStreams() | cli.dereference() == list(range(1, 6))

In [8]:
#export
def insertRow(*row:List[T]):
    """Inserts a row right before every other rows. See also: :meth:`joinList`."""
    return joinList(row)
def insertColumn(*column, begin=True, fillValue=""):
    """Inserts a column at beginning or end.
Example::

    # returns [['a', 1, 2], ['b', 3, 4]]
    [[1, 2], [3, 4]] | insertColumn("a", "b") | dereference()
"""
    return transpose(fillValue) | joinList(column, begin) | transpose(fillValue)

In [9]:
assert [[1, 2, 3], ["a", "b", "c"]] | insertRow(2, 3, 4) | cli.dereference() == [[2, 3, 4], [1, 2, 3], ['a', 'b', 'c']]
assert [[1, 2], [3, 4]] | insertColumn("a", "b") | cli.dereference() == [['a', 1, 2], ['b', 3, 4]]

In [10]:
#export
def insertIdColumn(table=False, begin=True, fillValue=""):
    """Inserts an id column at the beginning (or end).

:param table: if False, then insert column to an Iterator[str], else treat
    input as a full fledged table

Example::

    # returns [[0, 'a', 2], [1, 'b', 4]]
    [["a", 2], ["b", 4]] | insertIdColumn(True) | dereference()
    # returns [[0, 'a'], [1, 'b']]
    "ab" | insertIdColumn()"""
    f = (cli.toRange() & transpose(fillValue)) | joinList(begin=begin) | transpose(fillValue)
    if table: return f
    else: return cli.wrapList() | transpose() | f

In [11]:
assert [["a", 2], ["b", 4]] | insertIdColumn(True) | cli.dereference() == [[0, 'a', 2], [1, 'b', 4]]
assert "ab" | insertIdColumn() | cli.dereference() == [[0, 'a'], [1, 'b']]

In [12]:
#export
class toDict(BaseCli):
    def __init__(self, keyF:Callable[[Any], str]=None, valueF:Callable[[Any], Any]=None):
        """Transform an incoming stream into a dict using a function for
values. Example::

    names = ["wanda", "vision", "loki", "mobius"]
    names | toDict(valueF=lambda s: len(s)) # will return {"wanda": 5, "vision": 6, ...}
    names | toDict(lambda s: s.title(), lambda s: len(s)) # will return {"Wanda": 5, "Vision": 6, ...}
"""
        self.keyF = keyF or (lambda s: s)
        self.valueF = valueF or (lambda s: s)
    def __ror__(self, keys:Iterator[Any]) -> Dict[Any, Any]:
        keyF = self.keyF; valueF = self.valueF
        return {keyF(key):valueF(key) for key in keys}

In [13]:
assert ["wanda", "vision", "loki", "mobius"] | toDict(valueF=lambda s: len(s)) == {"wanda": 5, "vision": 6, "loki": 4, "mobius": 6}

In [14]:
#export
class split(BaseCli):
    def __init__(self, delim:str=None, idx:int=None):
        """Splits each line using a delimiter, and outputs the
parts as a separate line.

:param idx: if available, only outputs the element at that index"""
        self.delim = patchDefaultDelim(delim); self.idx = idx
    def __ror__(self, it:Iterator[str]):
        if self.idx == None:
            for line in it:
                for elem in line.split(self.delim): yield elem
        else:
            for line in it:
                elems = line.split(self.delim)
                yield elems[self.idx] if self.idx < len(elems) else None

In [15]:
#export
class table(BaseCli):
    def __init__(self, delim:str=None):
        """Splits lines to rows (List[str]) using a delimiter.
Example::

    # returns [['a', 'bd'], ['1', '2', '3']]
    ["a|bd", "1|2|3"] | table("|") | dereference()
"""
        self.delim = patchDefaultDelim(delim)
    def __ror__(self, it:Iterator[str]) -> Table[str]:
        return (line.split(self.delim) for line in it)
class stitch(BaseCli):
    def __init__(self, delim:str=None):
        """Stitches elements in a row together, so they become a simple string.
See also: :class:`k1lib.bioinfo.cli.output.pretty`"""
        self.delim = patchDefaultDelim(delim)
    def __ror__(self, it:Table[str]) -> Iterator[str]:
        d = self.delim
        for row in it: yield d.join(list(row))
def listToTable():
    """Turns Iterator[T] into Table[T]"""
    return cli.wrapList() | transpose()
tableFromList = listToTable

In [16]:
assert ["a|bd", "1|2|3"] | table("|") | cli.dereference() == [['a', 'bd'], ['1', '2', '3']]

In [17]:
#export
class count(BaseCli):
    """Finds unique elements and returns a table with [frequency, value, percent]
columns. Example::

    # returns [[1, 'a', '33%'], [2, 'b', '67%']]
    ['a', 'b', 'b'] | count() | dereference()
"""
    def __ror__(self, it:Iterator[str]):
        c = Counter(it); s = sum(c.values())
        for k, v in c.items(): yield [v, k, f"{round(100*v/s)}%"]

In [18]:
assert ['a', 'b', 'b'] | count() | cli.dereference() == [[1, 'a', '33%'], [2, 'b', '67%']]

In [19]:
#export
class permute(BaseCli):
    def __init__(self, *permutations:List[int]):
        """Permutes the columns. Acts kinda like :meth:`torch.Tensor.permute`"""
        self.permutations = permutations
    def __ror__(self, it:Iterator[str]):
        for row in it: yield (row[i] for i in self.permutations)

In [20]:
#export
class accumulate(BaseCli):
    def __init__(self, columnIdx:int=0, avg=False):
        """Groups lines that have the same row[columnIdx], and
add together all other columns, assuming they're numbers

:param columnIdx: common column index to accumulate
:param avg: calculate average values instead of sum"""
        self.columnIdx = columnIdx; self.avg = avg
        self.dict = defaultdict(lambda: defaultdict(lambda: 0))
    def __ror__(self, it:Iterator[str]):
        for row in it:
            row = list(row); key = row[self.columnIdx]
            row.pop(self.columnIdx)
            for i, e in enumerate(row):
                try: self.dict[key][i] += float(e)
                except: self.dict[key][i] = e
        for key, values in self.dict.items():
            n = len(self.dict[key].keys())
            if self.avg:
                for i in range(n):
                    if isinstance(self.dict[key][i], (int, float)):
                        self.dict[key][i] /= n
            elems = [str(self.dict[key][i]) for i in range(n)]
            elems.insert(self.columnIdx, key)
            yield elems

In [21]:
#export
class AA_(BaseCli):
    def __init__(self, *idxs:List[int], wraps=False):
        """Returns 2 streams, one that has the selected element, and the other
the rest. Example::

    [1, 5, 6, 3, 7] | AA_(1) # will return [5, [1, 6, 3, 7]]

You can also put multiple indexes through::

    [1, 5, 6] | AA_(0, 2) # will return [[1, [5, 6]], [6, [1, 5]]]

If you put None in, then all indexes will be sliced::

    [1, 5, 6] | AA_(0, 2)

    # will return:
    # [[1, [5, 6]],
    #  [5, [1, 6]],
    #  [6, [1, 5]]]

As for why the strange name, think of this operation as "AĀ". In statistics,
say you have a set "A", then "not A" is commonly written as A with an overline
"Ā". So "AA\_" represents "AĀ", and that it first returns the selection A
first.

:param wraps: if True, then the first example will return [[5, [1, 6, 3, 7]]]
    instead, so that A has the same signature as Ā
"""
        if len(idxs) == 0: raise AttributeError("Please enter some column indexes. If you wish to analyze all, put in `None`")
        self.idxs = idxs; self.wraps = wraps
    def __ror__(self, it:List[Any]) -> List[List[List[Any]]]:
        idxs = self.idxs; it = list(it)
        if len(idxs) == 1 and idxs[0] is None: idxs = range(len(it))
        def gen(idx):
            return [it[idx], [v for i, v in enumerate(it) if i != idx]]
        if not self.wraps and len(idxs) == 1: return gen(idxs[0])
        return [gen(idx) for idx in idxs]

In [22]:
#export
class sample(BaseCli):
    """Returns (firstRow, iterator). This sort of peaks at the first row, to
potentially gain some insights about the internal formats."""
    def __ror__(self, it):
        if (row := next(it, sentinel := object())) == sentinel: return None, []
        def gen(): yield row; yield from it
        return row, gen()

In [23]:
#export
class infinite(BaseCli):
    """Yields an infinite amount of the passed in object. Example:

.. code-block::

    # returns [[1, 2, 3], [1, 2, 3], [1, 2, 3]]
    [1, 2, 3] | infinite() | head(3) | toList()
"""
    def __ror__(self, o:T) -> Iterator[T]:
        try: o = list(o)
        except: pass
        while True: yield o

In [24]:
assert [1, 2, 3] | infinite() | cli.head(3) | cli.dereference() == [[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [27]:
!../../../export.py bioinfo/cli/structural

Current dir: /home/kelvin/repos/labs/k1lib, ../../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.10
Uninstalling k1lib-0.1.10:
  Successfully uninstalled k1lib-0.1.10
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1